In [1]:
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
plotly.offline.init_notebook_mode (connected = True)
import matplotlib.pyplot as plt
import warnings

In [2]:
header = 0 # 0 for No, 1 for Yes
label = 1 # 0 for No, 1 for Yes
path="iris.data.csv" #path to the datset
visualization=1 # 0 for No, 1 for Yes
apply = 1 # 0 for none, 1 for standardization, 2 for normalization

In [3]:
if header==1:
    df = pd.read_csv(path)
else:
    df = pd.read_csv(path,header=None)

if label==1:
    labeled=df[df.columns[-1]]
    df=df.drop([df.columns[-1]],axis=1)

In [4]:
df.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [5]:
#warnings.filterwarnings("ignore")

In [15]:
if visualization==1:
    df.corr().style.background_gradient(cmap='coolwarm') #to be worked upon for plotting by Shivesh

,0,1,2,3
0,1.000000,-0.109369,0.871754,0.817954
1,-0.109369,1.000000,-0.420516,-0.356544
2,0.871754,-0.420516,1.000000,0.962757
3,0.817954,-0.356544,0.962757,1.000000


In [7]:
feats = df.columns[:]
feats # getting the header names

Int64Index([0, 1, 2, 3], dtype='int64')

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

if apply==1:
    scaler = StandardScaler().fit_transform(df[feats])
elif apply==2:
    scaler = preprocessing.normalize(df[feats])
else:
    scaler = df[feats]

In [9]:
scaled = pd.DataFrame(scaler, columns=feats)
scaled

,0,1,2,3
0,-0.900681,1.032057,-1.341272,-1.312977
1,-1.143017,-0.124958,-1.341272,-1.312977
2,-1.385353,0.337848,-1.398138,-1.312977
3,-1.506521,0.106445,-1.284407,-1.312977
4,-1.021849,1.263460,-1.341272,-1.312977
...,...,...,...,...
145,1.038005,-0.124958,0.819624,1.447956
146,0.553333,-1.281972,0.705893,0.922064
147,0.795669,-0.124958,0.819624,1.053537
148,0.432165,0.800654,0.933356,1.447956


In [16]:
import plotly.express as px

if visualization==1:
    features = feats

    fig = px.scatter_matrix(
        scaled,
        dimensions=features
    )
    fig.update_traces(diagonal_visible=True)
    fig.show()

c:\Users\sjose\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

